In [3]:
import ee

# Only needed the very first time
ee.Authenticate()
ee.Initialize()

# Uganda boundary
uganda = ee.FeatureCollection("FAO/GAUL/2015/level0") \
           .filter(ee.Filter.eq('ADM0_NAME', 'Uganda'))

Enter verification code:  4/1Ab32j93qt5jlYN9WTTwXeSx79RTMfws7icIn4bEey3U2hEbqyrjcLqbAZ4s



Successfully saved authorization token.


In [4]:
# 1. GIMMS NDVI3g (1981–2015) — twice-monthly, reprojected to 0.083° (~8 km)

gimms = ee.ImageCollection('NASA/GIMMS/3GV0') \
          .select('ndvi') \
          .filterDate('1984-01-01', '2016-01-01')  # up to end of 2015

# Scale from -10000..10000 → -1..1 then to 0..1 for consistency
def scale_gimms(img):
    return img.multiply(0.0001).rename('NDVI') \
              .set('system:time_start', img.get('system:time_start'))

gimms_scaled = gimms.map(scale_gimms)

# Yearly means 1984–2015
years_gimms = ee.List.sequence(1984, 2015)
def yearly_gimms(y):
    y = ee.Number(y)
    start = ee.Date.fromYMD(y, 1, 1)
    end   = start.advance(1, 'year')
    mean = gimms_scaled.filterDate(start, end).mean() \
              .set('year', y) \
              .set('system:time_start', start.millis())
    return mean

gimms_yearly = ee.ImageCollection(years_gimms.map(yearly_gimms))

In [5]:
# 2. MODIS MOD13Q1 v006 (2000–2024) — 16-day 250 m

modis = ee.ImageCollection('MODIS/006/MOD13Q1') \
          .select('NDVI') \
          .filterDate('2016-01-01', '2025-01-01')

def scale_modis(img):
    return img.multiply(0.0001).rename('NDVI') \
              .set('system:time_start', img.get('system:time_start'))

modis_scaled = modis.map(scale_modis)

years_modis = ee.List.sequence(2016, 2024)
def yearly_modis(y):
    y = ee.Number(y)
    start = ee.Date.fromYMD(y, 1, 1)
    end   = start.advance(1, 'year')
    mean = modis_scaled.filterDate(start, end).mean() \
              .set('year', y) \
              .set('system:time_start', start.millis())
    return mean

modis_yearly = ee.ImageCollection(years_modis.map(yearly_modis))

C:\Users\Administrator\venv\Lib\site-packages\ee\deprecation.py:215: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
This dataset has been superseded by MODIS/061/MOD13Q1

Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


In [6]:
# 3. Combine both collections

ndvi_yearly_all = gimms_yearly.merge(modis_yearly)

# Clip to Uganda and add year as a band (optional)
def clip_and_label(img):
    year = img.get('year')
    return img.clip(uganda).set('year', year)

ndvi_yearly_uganda = ndvi_yearly_all.map(clip_and_label)

In [8]:
# === CORRECTED EXPORT LOOP (this works!) ===
years = range(1984, 2025)  # 1984 to 2024 inclusive

for year in years:
    image = ndvi_yearly_uganda \
              .filter(ee.Filter.eq('year', year)) \
              .first() \
              .select('NDVI')
    
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'Uganda_NDVI_{year}',
        folder='uganda_ndvi_yearly_1984_2024',
        fileNamePrefix=f'uganda_ndvi_{year}',
        region=uganda.geometry(),
        scale=1000,
        maxPixels=1e13,
        crs='EPSG:4326'
    )
    task.start()
    print(f"Started export for {year}")

print("\nAll 41 yearly NDVI exports are now running!")
print("Check your Google Drive in 20–60 minutes.")

Started export for 1984
Started export for 1985
Started export for 1986
Started export for 1987
Started export for 1988
Started export for 1989
Started export for 1990
Started export for 1991
Started export for 1992
Started export for 1993
Started export for 1994
Started export for 1995
Started export for 1996
Started export for 1997
Started export for 1998
Started export for 1999
Started export for 2000
Started export for 2001
Started export for 2002
Started export for 2003
Started export for 2004
Started export for 2005
Started export for 2006
Started export for 2007
Started export for 2008
Started export for 2009
Started export for 2010
Started export for 2011
Started export for 2012
Started export for 2013
Started export for 2014
Started export for 2015
Started export for 2016
Started export for 2017
Started export for 2018
Started export for 2019
Started export for 2020
Started export for 2021
Started export for 2022
Started export for 2023
Started export for 2024

All 41 yearly N

In [10]:
# CSV with national average

def reduce_year(img):
    mean = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=uganda.geometry(),
        scale=1000,
        maxPixels=1e10
    ).get('NDVI')
    
    return ee.Feature(None, {
        'year': img.get('year'),
        'NDVI_mean': mean
    })

table = ndvi_yearly_uganda.map(reduce_year).filter(ee.Filter.notNull(['NDVI_mean']))

task = ee.batch.Export.table.toDrive(
    collection=table,
    description='Uganda_yearly_mean_NDVI_1984_2024',
    folder='uganda_ndvi_yearly_1984_2024',
    fileFormat='CSV'
)
task.start()

print("Exporting CSV with yearly national average NDVI (1984–2024)...")

Exporting CSV with yearly national average NDVI (1984–2024)...
